# How Long Will The Bacterial Colony Last?

**2020-06-12**: https://fivethirtyeight.com/features/how-long-will-the-bacterial-colony-last/

## Riddler Express

From Dave Mattingly comes a high-rolling question of craps:

There’s a technique for rolling dice called “bowling,” in which you place your index finger and thumb on two opposite sides of the die and roll it along the table. If done correctly, the die will never land on the faces on which you were holding the die, leaving you with a 25 percent chance of landing on each of the remaining four faces.

You’d like to apply this technique to improve your chances of winning a simplified game of craps, in which your goal is simply to roll a 7 or 11 using two dice. With a standard rolling technique, your chances of rolling a 7 or 11 are 2/9, or about 22.2 percent.

Now suppose you’re using your bowling technique, and you roll the dice one at a time (i.e., you know the outcome of the first die before rolling the second). If you play to maximize your chances of rolling a 7 or 11, what will be your chances of winning?

### Solution

There are only a small number of outcomes, so let's just enumerate them and add the probabilities. 

First, let's consider the possible finger configurations for "bowling" style rolls. There are only 3 different configurations:

* Placing your fingers on the 1 and 6
* Placing your fingers on the 2 and 5
* Placing your fingers on the 3 and 6

This leads to the following 4 sides being possible rolls in each scenario.

```
{2, 3, 4, 5}
{1, 3, 4, 6}
{1, 2, 5, 6}
```

Based on the outcome of the first roll, you'll simply select a configuration that allows you to reach both the 7 and 11 total, or a 7 if it's not possible to reach both. 

```
1: {1, 3, 4, 6} or {1, 2, 5, 6} P(Win) = 25%
2: {2, 3, 4, 5} or {1, 2, 5, 6} P(Win) = 25%
3: {2, 3, 4, 5} or {1, 3, 4, 6} P(Win) = 25%
4: {2, 3, 4, 5} or {1, 3, 4, 6} P(Win) = 25%
5: {1, 2, 5, 6}                 P(Win) = 50%
6: {1, 2, 5, 6}                 P(Win) = 50%
```



If your first die is a 5 or 6 it becomes possible to reach both a 7 and 11, giving you a 50% probability of winning with the correct "bowling" throw. As a result, for our first throw let's choose the `{1, 2, 5, 6}` configuration to give ourselves a 50% chance of getting one of those numbers. Now we have a 50% chance of entering into a state with a 25% chance to win and a 50% chance of entering into a state with a 50% chance of winning.

\begin{equation*}
  \frac{1}{2} \times (\frac{1}{2} + \frac{1}{4} ) = \frac{3}{8} = 37.5\%
\end{equation*}

Let's confirm the result through simulation.

In [2]:
import numpy as np

n_sims = 10**6

# The bowling dictionary will take the result of the first roll as a key.
# The value is the possible outcomes of the bowling configuration we choose as a result of the first roll.
bowling = dict()
bowling[1] = [1, 3, 4, 6]
bowling[2] = [1, 2, 5, 6]
bowling[3] = [2, 3, 4, 5]
bowling[4] = [2, 3, 4, 5]
bowling[5] = [1, 2, 5, 6]
bowling[6] = [1, 2, 5, 6]

first_rolls = np.random.choice([1, 2, 5, 6], size=n_sims)

second_rolls = np.zeros(n_sims)

for i, roll in enumerate(first_rolls):
    second_rolls[i] = np.random.choice(bowling[roll])

totals = first_rolls + second_rolls

win_percentage = np.mean((totals == 7) | (totals == 11))
win_percentage

0.375073

This confirms our expected win percentage of `3/8`.

## Riddler Classic

From Adam Wagner comes a curious case of colonies:

You are studying a new strain of bacteria, Riddlerium classicum (or R. classicum, as the researchers call it). Each R. classicum bacterium will do one of two things: split into two copies of itself or die. There is an 80 percent chance of the former and a 20 percent chance of the latter.

If you start with a single R. classicum bacterium, what is the probability that it will lead to an everlasting colony (i.e., the colony will theoretically persist for an infinite amount of time)?

Extra credit: Suppose that, instead of 80 percent, each bacterium divides with probability `p`. Now what’s the probability that a single bacterium will lead to an everlasting colony?

### Solution

Unfortunately I've seen this type of question before, so I know it's very similar to [Bobo the Amoeba](https://www.quora.com/Bobo-the-amoeba-has-a-25-25-and-50-chance-of-producing-0-1-or-2-offspring-respectively-Each-of-Bobos-descendants-also-have-the-same-probabilities-What-is-the-probability-that-Bobos-lineage-dies-out) or the [Cave Escape](https://github.com/Booleans/math-through-simulation/blob/master/cave-escape.py). We simply set up an equation:

\begin{equation*}
  D = .20 + .80D^2
\end{equation*}

Where `D` represents the probability of a single bacterium dying off. It has a 20% chance of dying right away, but an 80% chance of transitioning to a state where 2 bacteria would have to die off. Expanding and solving the quadratic for `D` yields a 25% chance the bacterium dies out, meaning there's a 75% chance the colony lives on in perpetuity. Let's verify through simulation.


In [2]:
n_sims = 10**6
success = 0
failure = 0

for _ in range(n_sims):
    num_amoebas = 1
    while True:
        num_amoebas = np.sum(np.random.choice(np.array([0, 2]), p=[.2, .8], size=(num_amoebas)))
        # With such a high probability of reproduction I am choosing a threshold of 20 bacteria alive at once.
        # Above this threshold I assume the colony lives on in perpetuity. 
        if num_amoebas > 20:
            success += 1
            break
        elif num_amoebas == 0:
            failure += 1
            break
            
failure/n_sims

0.250062

This confirms that the colony dies out 25% of the time. 